In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

# Store F1 Data into a DataFrame

1. Original Data source 1 (FORMAT: CSV): Formula 1 race data - Formula 1 Race Data from 1950 to 2017 https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
    
Four csvs used:
1. circuits.csv: Contains a list of every formula 1 circuit, including name, location and geographic data.
2. drivers.csv: Contains a list of every Formula 1 driver, including full name, dob, and nationality.
3. races.csv: Details of every race, including year, date, time, circuit and round
4. results.csv: The results for each driver in each race.
5. constructors.csv: Contains a list of every constructor team including name and nationality.
6. constructorStandings.csv: Contains points, positions, and wins for each constructor.

In [2]:
# Read in F1 Grand Prix data from https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
races_csv = pd.read_csv("resources/f1_csvs/races.csv") 
results_csv = pd.read_csv("resources/f1_csvs/results.csv")
circuits_csv = pd.read_csv("resources/f1_csvs/circuits.csv")
drivers_csv = pd.read_csv("resources/f1_csvs/drivers.csv")
constructorStandings_csv = pd.read_csv("resources/f1_csvs/constructorStandings.csv")
constructors_csv = pd.read_csv("resources/f1_csvs/constructors.csv")

# Clean F1 Data

1. Get rid of unknown or uncommon races and only pull US races from circuits.csv and races.csv and then merge them
2. Merge drivers and results csvs on driver id
3. combine the two merged dataframes, drop unwanted columns, rename poorly named columns

In [3]:
# Set the index for the circuits csv to circuitId
circuits_csv = circuits_csv.set_index('circuitId')

# Locate only circuits in the US
circuits_csv = circuits_csv.loc[circuits_csv['country'] == 'USA']

# drop uncommon or unknown US circuits based on circuitId (Index)
circuits_csv = circuits_csv.drop([37, 42, 43, 44, 72])

# Reset the index to make merging easier later
circuits_csv = circuits_csv.reset_index()

circuits_csv.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...
1,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...
2,46,watkins_glen,Watkins Glen,New York State,USA,42.3369,-76.9272,NaN,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...
3,60,riverside,Riverside International Raceway,California,USA,33.9370,-117.2730,NaN,http://en.wikipedia.org/wiki/Riverside_Interna...
4,63,sebring,Sebring International Raceway,Florida,USA,27.4547,-81.3483,NaN,http://en.wikipedia.org/wiki/Sebring_Raceway


In [4]:
# Remove uncommon or unknown US circuits
races_csv = races_csv.loc[races_csv['circuitId'] != 43]

# Save only US Races
races_csv = races_csv.loc[races_csv['name'].isin(['United States Grand Prix', 'United States Grand Prix West'])]
races_csv.head()

,raceId,year,round,circuitId,name,date,time,url
41,42,2007,7,19,United States Grand Prix,6/17/2007,17:00:00,http://en.wikipedia.org/wiki/2007_United_State...
61,62,2006,10,19,United States Grand Prix,7/2/2006,14:00:00,http://en.wikipedia.org/wiki/2006_United_State...
78,79,2005,9,19,United States Grand Prix,6/19/2005,14:00:00,http://en.wikipedia.org/wiki/2005_United_State...
97,98,2004,9,19,United States Grand Prix,6/20/2004,NaN,http://en.wikipedia.org/wiki/2004_United_State...
121,122,2003,15,19,United States Grand Prix,9/28/2003,NaN,http://en.wikipedia.org/wiki/2003_United_State...


In [5]:
# Combine the circuits and races dfs
cir_race = pd.merge(circuits_csv, races_csv, on='circuitId')
cir_race = cir_race.drop(columns=['circuitRef', 'lat', 'lng', 'alt', 'url_x', 'url_y', 'name_y', 'time', 'url_y'])
cir_race = cir_race.rename(columns={'name_x':'trackName'})
cir_race.head()

,circuitId,trackName,location,country,raceId,year,round,date
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,7,6/17/2007
1,19,Indianapolis Motor Speedway,Indianapolis,USA,62,2006,10,7/2/2006
2,19,Indianapolis Motor Speedway,Indianapolis,USA,79,2005,9,6/19/2005
3,19,Indianapolis Motor Speedway,Indianapolis,USA,98,2004,9,6/20/2004
4,19,Indianapolis Motor Speedway,Indianapolis,USA,122,2003,15,9/28/2003


In [6]:
con_constand = pd.merge(constructors_csv, constructorStandings_csv, on='constructorId')
con_constand = con_constand.drop(columns=['constructorRef', 'url', 'positionText'])
con_constand.head()

,constructorId,name,nationality,constructorStandingsId,raceId,points,position,wins
0,1,McLaren,British,1,18,14.0,1,1
1,1,McLaren,British,7,19,24.0,1,1
2,1,McLaren,British,18,20,28.0,3,1
3,1,McLaren,British,29,21,34.0,3,1
4,1,McLaren,British,40,22,42.0,3,1


In [7]:
# Combine the drivers and the results dfs
dri_res = pd.merge(drivers_csv, results_csv, on='driverId')
dri_res = dri_res.drop(columns=['driverRef', 'number_x', 'code', 'dob', 'url', 'milliseconds', 'statusId', 'constructorId'])
dri_res.head()

,driverId,forename,surname,nationality,resultId,raceId,number_y,grid,position,positionText,positionOrder,points,laps,time,fastestLap,rank,fastestLapTime,fastestLapSpeed
0,1,Lewis,Hamilton,British,1,18,22.0,1,1.0,1,1,10.0,58,34:50.6,39.0,2.0,01:27.5,218.3
1,1,Lewis,Hamilton,British,27,19,22.0,9,5.0,5,5,4.0,56,46.548,53.0,3.0,01:35.5,209.033
2,1,Lewis,Hamilton,British,57,20,22.0,3,13.0,13,13,0.0,56,NaN,25.0,19.0,01:35.5,203.969
3,1,Lewis,Hamilton,British,69,21,22.0,5,3.0,3,3,6.0,66,4.187,20.0,3.0,01:22.0,204.323
4,1,Lewis,Hamilton,British,90,22,22.0,3,2.0,2,2,8.0,58,3.779,31.0,2.0,01:26.5,222.085


In [8]:
con_res = pd.merge(dri_res, con_constand, on='raceId')
con_res = con_res.drop(columns=['number_y', 'grid', 'positionText', 'positionOrder'])
con_res = con_res.rename(columns={'nationality_x':'driverNationality',
                                  'position_x': 'driverPosition',
                                  'nationality_y': 'constructorNationality',
                                  'points_x': 'driverPoints',
                                  'points_y': 'constructorPoints',
                                  'position_y': 'constructorPosition',
                                  'forename': 'firstName',
                                  'wins': 'constructorWins'})
con_res.head()

,driverId,firstName,surname,driverNationality,resultId,raceId,driverPosition,driverPoints,laps,time,...,rank,fastestLapTime,fastestLapSpeed,constructorId,name,constructorNationality,constructorStandingsId,constructorPoints,constructorPosition,constructorWins
0,1,Lewis,Hamilton,British,1,18,1.0,10.0,58,34:50.6,...,2.0,01:27.5,218.3,1,McLaren,British,1,14.0,1,1
1,1,Lewis,Hamilton,British,1,18,1.0,10.0,58,34:50.6,...,2.0,01:27.5,218.3,2,BMW Sauber,German,2,8.0,3,0
2,1,Lewis,Hamilton,British,1,18,1.0,10.0,58,34:50.6,...,2.0,01:27.5,218.3,3,Williams,British,3,9.0,2,0
3,1,Lewis,Hamilton,British,1,18,1.0,10.0,58,34:50.6,...,2.0,01:27.5,218.3,4,Renault,French,4,5.0,4,0
4,1,Lewis,Hamilton,British,1,18,1.0,10.0,58,34:50.6,...,2.0,01:27.5,218.3,5,Toro Rosso,Italian,5,2.0,5,0


In [9]:
# combine the circuits/race df and the driver/results df into one final combined df
combined_df = pd.merge(cir_race, con_res, on='raceId')

combined_df = combined_df.drop(columns=['round'])

# Rename columns to something more meaningful
combined_df = combined_df.rename(columns={'location':'raceLocation',
                                          'country': 'raceCountry',
                                          'year': 'raceYear',
                                          'date': 'raceDate',
                                          'firstName': 'driverFirstName',
                                          'surname': 'driverSurname',
                                          'laps': 'driverLaps',
                                          'time': 'driverTime',
                                          'fastestLap': 'driverFastestLap',
                                          'rank': 'driverRank',
                                          'fastestLapTime': 'driverFastestLapTime',
                                          'fastestLapSpeed':'driverFastestLapSpeed',
                                          'name':'constructorName'})
combined_df.head()

,circuitId,trackName,raceLocation,raceCountry,raceId,raceYear,raceDate,driverId,driverFirstName,driverSurname,...,driverRank,driverFastestLapTime,driverFastestLapSpeed,constructorId,constructorName,constructorNationality,constructorStandingsId,constructorPoints,constructorPosition,constructorWins
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,1,McLaren,British,25800,106.0,11,4
1,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,2,BMW Sauber,German,25801,39.0,2,0
2,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,3,Williams,British,25803,13.0,4,0
3,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,4,Renault,French,25802,25.0,3,0
4,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,5,Toro Rosso,Italian,25808,0.0,8,0


# Store Weather Data into a DataFrame

2. Original Data source 2 (FORMAT: NOT YET KNOWN): Weather data is the Local Climatologcial Data from NOAA.gov - https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd

In [10]:
# Create dataframe from csv
austin_weather = pd.read_csv('resources/weather_csvs/austin_weather.csv')
indy_weather = pd.read_csv('resources/weather_csvs/indy_weather.csv')

C:\Users\sengeling\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (33,34,41,42,43,46,49,56,73,75,76,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\sengeling\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31,33,34,41,42,43,46,48,49,51,52,53,56,58,59,60,61,63,69,71,73,75,76,77,82,88,89,95,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Clean Weather Data

In [11]:
# Keep only the rows with code type 'SOD' (indicating daily weather data) in the 'REPORT_TYPE' column
austin_weather = austin_weather.loc[austin_weather['REPORT_TYPE'].str.strip() == 'SOD']

# Drop all columns that have all NaN values
austin_weather.dropna(axis=1, how='all', inplace=True)

# Add column that clarifies the city name
austin_weather['city'] = 'Austin, TX'

# From the remaining columns, keep the ones specified
austin_weather = austin_weather[['STATION', 'DATE', 'city', 'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity', 
                          'DailyAverageStationPressure', 'DailyAverageWindSpeed', 'DailyMaximumDryBulbTemperature', 
                          'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection', 'DailyPrecipitation']]

# Rename columns, reset index
austin_weather.columns = ['station', 'date', 'city', 'avg_temp', 'avg_rel_humidity', 'avg_pressure', 'avg_wind', 'max_temp',
                         'min_temp', 'peak_wind_dir', 'precip']

austin_weather.reset_index(drop=True, inplace=True)

# Clean the 'date' column by keeping only the date ('YYYY-MM-DD') and dropping extra info (time entered)
austin_weather['date'] = [austin_weather['date'][x][:10] for x in range(len(austin_weather['date']))]

# Instantiate list of dates
austin_dates = ['2012-11-18', '2013-11-17', '2014-11-02', '2015-10-25', '2016-10-23', '2017-10-22', '2018-10-21']

# Keep only rows whose date is in the list austin_dates and reset the index again
austin_weather = austin_weather.loc[austin_weather['date'].isin(austin_dates)]

austin_weather.reset_index(drop=True, inplace=True)

# Show dataframe
austin_weather

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72254013904,2012-11-18,"Austin, TX",55.0,65.0,29.70,4.6,74.0,36.0,160.0,0.00
1,72254013904,2013-11-17,"Austin, TX",75.0,78.0,29.33,5.0,89.0,61.0,200.0,0.00
2,72254013904,2014-11-02,"Austin, TX",57.0,64.0,29.68,8.6,76.0,38.0,150.0,0.00
3,72254013904,2015-10-25,"Austin, TX",62.0,88.0,29.47,16.7,65.0,58.0,350.0,0.57
4,72254013904,2016-10-23,"Austin, TX",66.0,68.0,29.62,5.6,84.0,47.0,180.0,0.01
5,72254013904,2017-10-22,"Austin, TX",64.0,81.0,29.57,8.2,78.0,50.0,330.0,0.70
6,72254013904,2018-10-21,"Austin, TX",65.0,61.0,29.70,9.6,74.0,55.0,20.0,0.00


In [12]:
# Keep only the rows with code type 'SOD' (indicating daily weather data) in the 'REPORT_TYPE' column
indy_weather = indy_weather.loc[indy_weather['REPORT_TYPE'].str.strip() == 'SOD']

# Drop all columns that have all NaN values
indy_weather.dropna(axis=1, how='all', inplace=True)

# Add column that clarifies the city name
indy_weather['city'] = 'Indianapolis, IN'

# From the remaining columns, keep the ones specified
indy_weather = indy_weather[['STATION', 'DATE', 'city', 'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity', 
                          'DailyAverageStationPressure', 'DailyAverageWindSpeed', 'DailyMaximumDryBulbTemperature', 
                          'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection', 'DailyPrecipitation']]

# Rename columns, reset index
indy_weather.columns = ['station', 'date', 'city', 'avg_temp', 'avg_rel_humidity', 'avg_pressure', 'avg_wind', 'max_temp',
                         'min_temp', 'peak_wind_dir', 'precip']

indy_weather.reset_index(drop=True, inplace=True)

# Clean the 'date' column by keeping only the date ('YYYY-MM-DD') and dropping extra info (time entered)
indy_weather['date'] = [indy_weather['date'][x][:10] for x in range(len(indy_weather['date']))]

# Instantiate list of dates
indy_dates = ['2000-09-24', '2001-09-30', '2002-09-29', '2003-09-28', '2004-06-20', '2005-06-19', 
              '2006-07-02', '2007-06-17']

# Keep only rows whose date is in the list austin_dates and reset the index again
indy_weather = indy_weather.loc[indy_weather['date'].isin(indy_dates)]

indy_weather.reset_index(drop=True, inplace=True)

# Show dataframe
indy_weather

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,2000-09-24,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,2001-09-30,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,2002-09-29,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,2003-09-28,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,2004-06-20,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s
5,72438093819,2005-06-19,"Indianapolis, IN",67,NaN,29.26,8.1,77,57,040,0.00
6,72438093819,2006-07-02,"Indianapolis, IN",80,63.0,29.21,10.0,90,70,270,0.00
7,72438093819,2007-06-17,"Indianapolis, IN",82,49.0,29.15,7.8,93,71,260,T


# Combine the Weather DataFrames
Commented out as a failsafe.

In [13]:
# Save dataframe to a csv file
# indy_weather.to_csv('weather_csvs/weather_data.csv', index=False)

In [14]:
# Append the other dataframe to the weather_data csv
# with open('weather_csvs/weather_data.csv', 'a') as file:
#     austin_weather.to_csv(file, header=False, index=False)

In [15]:
# Create dataframe from the weather_data csv
combined_weather = pd.read_csv('resources/weather_csvs/weather_data.csv')

combined_weather.head()

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,9/24/2000,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,9/30/2001,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,9/29/2002,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,9/28/2003,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,6/20/2004,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s


# Create Database
Done in Stephanie's Workbench - f1_weather_db.

# Connect to Local

Relational database, chosen in order to join two different types of data on a common key - date

In [16]:
# Connect to the database
rds_connection_string = "root:Stup!dR00tP@55w0rd@127.0.0.1/f1_weather_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Use Pandas to Load Race Data into Database as a Table
f1_data


In [18]:
# Add the combined f1 dataframe to the database
combined_df.to_sql(name='f1_data', con=engine, if_exists='append', index=False)

# Use Pandas to Load Weather Data into Database as a Table
weather_data

In [19]:
combined_weather.to_sql(name='weather_data', con=engine, if_exists='append', index=False)

# Confirm Data Has Been Loaded

In [20]:
# Test that the f1 table is presnt and working
pd.read_sql_query('select * from f1_data', con=engine).head()

,circuitId,trackName,raceLocation,raceCountry,raceId,raceYear,raceDate,driverId,driverFirstName,driverSurname,...,driverRank,driverFastestLapTime,driverFastestLapSpeed,constructorId,constructorName,constructorNationality,constructorStandingsId,constructorPoints,constructorPosition,constructorWins
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,1,McLaren,British,25800,106.0,11,4
1,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,2,BMW Sauber,German,25801,39.0,2,0
2,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,3,Williams,British,25803,13.0,4,0
3,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,4,Renault,French,25802,25.0,3,0
4,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,...,2.0,01:13.2,206.101,5,Toro Rosso,Italian,25808,0.0,8,0


In [21]:
# Test that the weather table is presnt and working
pd.read_sql_query('select * from weather_data', con=engine).head()

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,9/24/2000,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,9/30/2001,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,9/29/2002,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,9/28/2003,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,6/20/2004,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s


# Test a Join on Date

In [24]:
pd.read_sql_query('select    f1_data.trackName,\
                             f1_data.raceLocation,\
                             f1_data.raceDate,\
                             f1_data.driverFirstName,\
                             f1_data.driverSurname,\
                             f1_data.driverPosition,\
                             f1_data.driverPoints,\
                             f1_data.driverRank,\
                             f1_data.driverFastestLapTime,\
                             f1_data.driverFastestLapSpeed,\
                             f1_data.constructorName,\
                             f1_data.constructorNationality,\
                             f1_data.constructorPoints,\
                             f1_data.constructorPosition,\
                             f1_data.constructorWins,\
                             weather_data.date,\
                             weather_data.avg_temp,\
                             weather_data.avg_rel_humidity,\
                             weather_data.avg_wind,\
                             weather_data.peak_wind_dir,\
                             weather_data.precip\
                    from     f1_data, weather_data\
                    where    f1_data.raceDate = weather_data.date\
                    group by f1_data.driverSurname, f1_data.raceDate\
                    order by f1_data.raceDate, f1_data.driverPosition;', con=engine)


,trackName,raceLocation,raceDate,driverFirstName,driverSurname,driverPosition,driverPoints,driverRank,driverFastestLapTime,driverFastestLapSpeed,...,constructorNationality,constructorPoints,constructorPosition,constructorWins,date,avg_temp,avg_rel_humidity,avg_wind,peak_wind_dir,precip
0,Circuit of the Americas,Austin,10/22/2017,Fernando,Alonso,NaN,0.0,18.0,01:41.5,195.463,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
1,Circuit of the Americas,Austin,10/22/2017,Nico,HÌ_lkenberg,NaN,0.0,19.0,01:44.3,190.34,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
2,Circuit of the Americas,Austin,10/22/2017,Daniel,Ricciardo,NaN,0.0,9.0,01:40.1,198.265,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
3,Circuit of the Americas,Austin,10/22/2017,Pascal,Wehrlein,NaN,0.0,20.0,01:47.1,185.357,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
4,Circuit of the Americas,Austin,10/22/2017,Lewis,Hamilton,1.0,25.0,5.0,01:38.8,200.927,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
5,Circuit of the Americas,Austin,10/22/2017,Sebastian,Vettel,2.0,18.0,6.0,01:38.8,200.86,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
6,Circuit of the Americas,Austin,10/22/2017,Kimi,RÌ_ikkÌ¦nen,3.0,15.0,1.0,01:37.8,203.003,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
7,Circuit of the Americas,Austin,10/22/2017,Max,Verstappen,4.0,12.0,4.0,01:38.1,202.394,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
8,Circuit of the Americas,Austin,10/22/2017,Valtteri,Bottas,5.0,10.0,2.0,01:37.8,203.001,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7
9,Circuit of the Americas,Austin,10/22/2017,Esteban,Ocon,6.0,8.0,13.0,01:40.5,197.482,...,British,23.0,9,0,10/22/2017,64,81.0,8.2,330,0.7


# Future Considerations

We missed the opportunity to pull constructor (car brand) down and include it in the database for interesting analysis against weather.

In [22]:
combined_df.columns

Index(['circuitId', 'trackName', 'raceLocation', 'raceCountry', 'raceId',
       'raceYear', 'raceDate', 'driverId', 'driverFirstName', 'driverSurname',
       'driverNationality', 'resultId', 'driverPosition', 'driverPoints',
       'driverLaps', 'driverTime', 'driverFastestLap', 'driverRank',
       'driverFastestLapTime', 'driverFastestLapSpeed', 'constructorId',
       'constructorName', 'constructorNationality', 'constructorStandingsId',
       'constructorPoints', 'constructorPosition', 'constructorWins'],
      dtype='object')